In [8]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("/Users/oliver/Desktop/mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/oliver/Desktop/mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/oliver/Desktop/mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/oliver/Desktop/mnist/data/t10k-images-idx3-ubyte.gz
Extracting /Users/oliver/Desktop/mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


导入tensorflow

In [9]:
import tensorflow as tf

通过操作符号变量来描述可交互的操作单元

In [10]:
# x不是一个特定的值，而是一个占位符placeholder
# 我们在Tensorflow运行计算时输入这个值
# 我们希望能够输入任何数量的MNIST图像
# 每一张图展成784维的向量
# 我们用2维的浮点数张量来表示这些图
# 这个张量的形状是[None, 784]  (这里的None表示此张量的第一个维度可以是任何长度的)
x = tf.placeholder(tf.float32, [None, 784])

参数初始化

In [11]:
# 对于权重和偏置量，tensorflow有更好的方法表示Variable
# 一个Variable代表一个可修改的张量
# 它们可以用于计算输入值，也可以在计算中被修改
W = tf.Variable(tf.zeros([784, 10]))
B = tf.Variable(tf.zeros([10]))

# W的维度是[784, 10]，因为我们想要用784维的图片向量乘以它
# 用来得到一个10维的证据值向量
# 每一位对应不同的数字类
# b的形状是[10]，所以我们可以直接把它加到输出上面

实现模型

In [12]:
y = tf.nn.softmax(tf.matmul(x, W) + B)

# 这里x是一个2维张量拥有多个输入。

定义成本(cost)或者说损失(loss)

In [13]:
y_ = tf.placeholder("float", [None, 10])

计算交叉熵
    $- \sum y^{'} log(y)$

In [14]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))  
# reduce表示降维的意思，在这里就是降维求和

反向传播(backpropagation) 以及梯度下降

In [15]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

ok，在上面我们已经设置好了我们的模型。
在运算之前我们需要添加一个操作来初始化我们的创建变量

In [16]:
init = tf.global_variables_initializer()

在一个Session里面启动我们的模型，并且初始化变量

In [17]:
sess = tf.Session()
sess.run(init)

开始训练模型，这里我们让模型循环训练1000次

In [18]:
for i in range(1000) :
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})

# 该循环的每个步骤中，我们都会随机抓去训练数据中的100个批处理数据点
# 然后我们用这些数据点作为参数替换占位符来运行train_step

- 那么我们模型的性能如何？
- 首先需要让我们找出那些预测正确的标签
- tf.argmax是一个非常有用的函数，它能给出某个tensor对象在某一维上的其数据最大值所在的索引值
- 然后我们可以用tf.equal来检测我们的预测是否于真实标签匹配（索引位置一样表示匹配）

In [19]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# 这行代码会给一组布尔值。
# 为了确定正确预测项的比例
# 我们可以把布尔值转换成浮点数，然后取平均值
# 例如[True, False, True, True]会变成[1, 0, 1, 1]
# 取得平均值后得到0.75

- tf.argmax(input,dimension)  
    - dimension = 0 按列找最大值
    - dimension =1 按行找最大值

In [20]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

最后，我们计算所学习到的模型在测试数据集上面的正确率

In [21]:
print( sess.run(accuracy, feed_dict={x: mnist.test.images, y_:mnist.test.labels}) )

0.9178


# 卷积神经网络
- 权重和偏置初始化
    - 权重在初始化时应该加入少量的噪声来打破对称性以及避免0梯度
    - 由于我们使用的ReLU神经元，因此比较好的做法就是用一个较小的正数来初始化偏置项，避免神经元节点输出恒为0
    - 为了不再建立模型的时候反复做初始化操作，我们定义了两个函数用于初始化

In [35]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("/Users/oliver/Desktop/mnist/data/", one_hot=True)

Extracting /Users/oliver/Desktop/mnist/data/train-images-idx3-ubyte.gz
Extracting /Users/oliver/Desktop/mnist/data/train-labels-idx1-ubyte.gz
Extracting /Users/oliver/Desktop/mnist/data/t10k-images-idx3-ubyte.gz
Extracting /Users/oliver/Desktop/mnist/data/t10k-labels-idx1-ubyte.gz


In [36]:
def weight_variable(shape) :
    initial = tf.truncated_normal(shape, stddev=0)
# tf.truncated_normal(shape, mean, stddev) 
# shape表示生成张量的维度，mean是均值，stddev是标准差
# 这个函数产生正太分布，均值和标准差自己设定。
# t输出如字面意思是截断的，而截断的标准是2倍的stddev。
# 截断的意思就是举例，当输入参数mean = 0 ，stddev =1时，
# 使用tf.truncated_normal的输出是不可能出现[-2,2]以外的点的，
    return tf.Variable(initial)

def bias_variable(shape) :
    initial = tf.constant(0.1, shape=shape)
# 创建一个常数张量,传入list或者数值来填充
    return tf.Variable(initial)

## 池化和卷机
- tensorflow在卷积和池化上有很强的灵活性
- 我们怎么处理边界？步长应该设多大？
- 我们的卷积使用1步长(stride size)
- 我们的卷积使用0边距(padding size)
- 通过上述设定保证输出和输入时同一个大小
- 我们的池化用简单传统的2x2大小的模版做max pooling
- 为了代码更加简洁，我们把这部分抽象成一个函数

In [37]:
def conv2d(x, W) : 
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x) : 
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                                     strides=[1, 2, 2, 1], padding='SAME')

## 第一层卷机
- 卷积在每个5x5的patch中算出32个特征
- 卷积的权重张量形状时[5, 5, 1, 32]
    - 前两个维度时patch的大小，接着是输入的通道数目，最后是输出的通道数目
- 而对于每一个输出通道都有一个对应的偏置量

In [38]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [39]:
# 为了用到这一层，我们把x变成一个4dimension向量
# 其第2，3维对应图片的宽，高。
# 最后一维对应图片的颜色通道数
x_image = tf.reshape(x,[-1, 28, 28, 1])

In [40]:
# 然后我们把x_image和权值向量进行卷积，加上偏置项
# 然后应用ReLU激活函数，最后进行max pooling
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## 第二层卷机
- 为了构建一个更深的网络，我们会把几个类似的层堆叠起来
- 第二层中，每个5x5的patch会得到64个特征

In [41]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)                

## 密集连接层
- 现在，图片尺寸减小到7x7
- 我们加入一个有1024个神经元的全连接层，用于处理整个图片

In [42]:
# 我们把池化层输出的张量reshape成一些向量
# 乘上权重矩阵，加上偏置
# 然后对其使用ReLU
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout
- 为了减少过拟合，我们在输出层之前加入dropout。
- 我们用一个placeholder来带遍一个神经元的输出在dropout中保持不变的概率
- 这样我们可以在训练过程中启用dropout，在测试过程中关闭dropout

In [43]:
# TensorFlow的tf.nn.dropout操作
# 除了可以屏蔽神经元的输出外
# 还会自动处理神经元输出值的scale
# 所以用dropout的时候可以不用考虑scale
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 输出层
- 最后我们添加一个softmax层，就像前面的单层softmax regression一样

In [53]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## 训练和评估模型
- 为了进行训练和评估，我们使用与之前简单的单层softmax神经网络模型几乎相同的一套代码
- 只是我们会用更加复杂的ADAM优化器来做梯度最速下降
- 在feed_dict中加入额外的参数keep_prob来控制dropout比例
- 最后每100次迭代输出一次日志

In [ ]:
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    for i in range(20000) :
        batch = mnist.train.next_batch(50)
        if i%100 == 0 : 
            train_accuracy = accuracy.eval(feed_dict={x : batch[0], y_ : batch[1], keep_prob : 1.0})
            print("step %d，training accuracy %g" % (i, train_accuracy))

        train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

    print("test accuracy %g" % accuracy.eval(feed_dict={
        x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

step 0，training accuracy 0.18
step 100，training accuracy 0.1
step 200，training accuracy 0.08
step 300，training accuracy 0.26
step 400，training accuracy 0.6
step 500，training accuracy 0.78
step 600，training accuracy 0.78
step 700，training accuracy 0.94
step 800，training accuracy 0.94
step 900，training accuracy 0.96
step 1000，training accuracy 0.94
step 1100，training accuracy 0.9
step 1200，training accuracy 0.96
step 1300，training accuracy 0.94
step 1400，training accuracy 0.98
step 1500，training accuracy 0.98
step 1600，training accuracy 0.94
step 1700，training accuracy 0.96
step 1800，training accuracy 0.96
step 1900，training accuracy 0.96
step 2000，training accuracy 0.96
step 2100，training accuracy 0.96
step 2200，training accuracy 0.98
step 2300，training accuracy 0.98
step 2400，training accuracy 1
step 2500，training accuracy 0.96
step 2600，training accuracy 0.96
step 2700，training accuracy 0.92
step 2800，training accuracy 0.96
step 2900，training accuracy 1
step 3000，training accuracy 1
s